In this notebook, we will show how to run Apps in SigPy.

For a complete list of MRI Apps, please see: https://sigpy.readthedocs.io/en/latest/mri_app.html

# Setup

In addition to Numpy and SigPy, we will need to import the mri submodule `sigpy.mri` for MRI apps.

In [ ]:
%matplotlib notebook
import numpy as np
import sigpy as sp
import sigpy.mri as mr
import sigpy.plot as pl

We will be using a 8x undersampled 8-channel brain dataset:

In [ ]:
ksp = np.load('data/brain_ksp.npy')
ksp /= np.abs(sp.ifft(ksp, axes=[-1, -2])).max()

pl.ImagePlot(ksp)

A root-sum-of-squares reconstruction looks like this:

In [ ]:
img_rss = np.sum(np.abs(sp.ifft(ksp, axes=(-1, -2)))**2, axis=0)**0.5

pl.ImagePlot(img_rss)

## Estimating sensitivity maps using ESPIRiT

We will use ESPIRiT to compute the sensitivity maps. This is provided in SigPy as `mr.espirit_maps`. This function supports GPU with the option `device`.

In [ ]:
mps = mr.espirit_maps(ksp, calib_width=24, kernel_width=8)

pl.ImagePlot(mps)

## SENSE Recon

We will first run the SENSE App. To run an App, simply do, `app.run()`.

All MRI reconstruction Apps support the device option, so they can be run on GPU as well.

In [ ]:
img_sense = mr.app.SenseRecon(ksp, mps, lamda=0.01).run()

pl.ImagePlot(img_sense)

## L1 Wavelet Recon

The L1-wavelet App does the l1-wavelet reconstruction.

In [ ]:
lamda = 0.005
img_l1wav = mr.app.L1WaveletRecon(ksp, mps, lamda, device=0).run()

pl.ImagePlot(img_l1wav)

# Total Variation Recon

And we can do total variation regularized reconstruction.

In [ ]:
lamda = 0.005
img_tv = mr.app.TotalVariationRecon(ksp, mps, lamda).run()

pl.ImagePlot(img_tv)